In [1]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

In [3]:
# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
json.loads(ret)

{'result': [{'url': 'https://ekitan.com/event/station-2590',
   'title': '東京駅周辺のイベント - 駅探',
   'content': '## 東京駅のイベント一覧. ### 条件指定. * #### 日本の桜名所100選\u3000赤城南面千本桜！菜の花・桜・花桃の中を駆け抜けるトロッコわたらせ渓谷号. + 二重橋前駅／東京駅／大手町駅(東京)駅. * #### 平日昼間\u3000初心者向け 山手線1周ラン 約10～42キロ（キロ約7～8分）. + 東京駅／二重橋前駅／京橋駅(東京)駅. + 二重橋前駅／東京駅／大手町駅(東京)駅. * #### 山手線1周ウォーク 約10～42キロ（時速3～4キロ） 11月. + 東京駅／二重橋前駅／京橋駅(東京)駅. * #### 山手線1周ウォーク 約10～42キロ（時速3～4キロ） 5月. + 東京駅／二重橋前駅／京橋駅(東京)駅. * #### 初心者向け 山手線1周ラン 約10～42キロ（キロ約7～8分） 4月. + 東京駅／二重橋前駅／京橋駅(東京)駅. * #### 初心者向け 山手線1周ラン 約10～42キロ（キロ約7～8分） 4月. + 東京駅／二重橋前駅／京橋駅(東京)駅. * #### 初心者向け ゲートブリッジラン 約15～60キロ（キロ約7～8分）. + 東京駅／二重橋前駅／京橋駅(東京)駅. ### 近くの駅のイベント一覧. ### 東京駅沿線のイベント一覧. ### 東京駅の周辺情報. ### 東京駅の路線図・停車駅. ### 東京駅の運行情報. ### 東京駅発の人気の区間(乗換案内). ### 関連サービス. ### **おすすめ記事**.',
   'score': 0.8170061,
   'raw_content': None},
  {'url': 'https://iko-yo.net/events?prefecture_ids%5B%5D=13&tags%5B%5D=%E6%9D%B1%E4%BA%AC%E9%A7%85&term=1',
   'title': '東京都 東京駅の 子ども・親子おでかけ 今日のイベント情報 - いこーよ',
   'content': '# 東京都（東京駅）のイベント [

In [4]:
# ツール定義
def define_tools():
    print("------define_tools(ツール定義)------")
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "最近一ヵ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

In [5]:
# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response

In [6]:
# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

In [7]:
# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [8]:
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
東京都と沖縄県の面積を比較すると、沖縄県の方が広いです。

- **東京都**の面積は約2,194平方キロメートルです。
- **沖縄県**の面積は約2,271平方キロメートルです。

したがって、沖縄県の方が東京都よりも広いです。


In [9]:
tools = define_tools()

# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
最近1ヶ月以内に東京駅でのイベントについての情報をいくつかご紹介します。

1. **[東京駅周辺のイベント一覧](https://bestcalendar.jp/events/%E6%9D%B1%E4%BA%AC%E9%A7%85)**
   - 東京駅周辺での最新のイベント情報や開催日スケジュールを紹介しています。様々な地元の祭りや特別企画があるようです。

2. **[Let's ENJOY TOKYO](https://www.enjoytokyo.jp/event/list/area1306/)**
   - 東京駅とその周辺で行われるイベントやアクティビティのリストが掲載されています。参加予定のイベントや興味深い企画があれば、是非チェックしてみてください。

3. **[東京のイベント情報](https://www.enjoytokyo.jp/event/list/)**
   - 東京全体のイベントを網羅したサイトで、東京駅周辺のイベントも紹介しています。最新の開催情報や内容が詳しく記載されています。

4. **[GranSta Tokyoの新着情報](https://www.gransta.jp/news/)**
   - 東京駅構内のショッピングやグルメスポット「GranSta」に関する新着情報が掲載されています。新しいショップのオープン情報等も含まれています。

5. **[東京イベント情報](https://www.tokyofes.info/)**
   - 東京に関する最新のイベントやフェスティバル情報を提供しているサイトです。週末のイベントや特別な提案が多数あります。

興味のあるイベントがあれば、ぜひ訪れてみてください！


In [10]:
# チャットボットへの組み込み
tools = define_tools()

messages=[]

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        del_message = messages.pop(0)

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

------define_tools(ツール定義)------


'質問:こんにちは！'

こんにちは！今日はどんなことをお手伝いできますか？


'質問:東北6県は？'

東北地方は日本の北東部に位置しており、以下の6つの県から構成されています。

1. 青森県
2. 岩手県
3. 宮城県
4. 秋田県
5. 山形県
6. 福島県

これらの県は、豊かな自然や文化を持ち、それぞれに特徴があります。


'質問:宮城県のお土産について検索した結果を教えて'

宮城県には多くの特徴的なお土産があります。以下に、いくつかの代表的なお土産を紹介します。

1. **ずんだ餅**: ずんだ（枝豆のペースト）を使った餅で、特に人気があります。甘さと塩気のバランスが絶妙です。

2. **他」のもっこりプリン**: 仙台名物のプリンは、バニラ風味や抹茶風味など多様なバリエーションがあります。

3. **萩の月**: 仙台の有名な洋菓子で、ふわふわの生地とクリームが入ったお菓子です。見た目も美しく、お土産に最適です。

4. **仙台牛**: お肉好きにはたまらない、仙台牛の肉製品も人気のお土産です。焼き肉やお弁当に役立ちます。

5. **笹かまぼこ**: 国産の新鮮な魚を使った伝統的なかまぼこで、特有の食感と味わいがあります。

これらの情報は、宮城県の観光やお土産に関する情報が集約されたウェブサイトや記事をもとにまとめました。具体的なお土産や購入場所についての詳細は、各リンク先を参照してください。

---ご利用ありがとうございました！---
